# Information Retrieval on Verses of the Holy Quran
Team Labs 2022 : 
Mohammed Abdul Khaliq,
Ufkun Bayram Menderes,
Muhammad Saad Magdi

In [64]:
import warnings
warnings.filterwarnings('ignore')
import math

import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
import numpy as np
from nltk.stem import PorterStemmer

import collections
from collections import Counter
import more_itertools

import time

import nltk 
import regex as re



#IMPORTANT VARIABLES TO TINKER
subset_size = 1137 #the amount of rows to be taken from the humongous dataset (0 - for full dataset)
samples_invindex = 7 #the amount of samples of the inverted index to display (0 - for full inverted index)
samples_postlist = 7 #the amount of samples of postings list to display (0 - for full posting list)
samples_intersect = 7 #the amount of text samples of intersection to display (0 - all intersected tweets)




#nltk.download('punkt')
#nltk.download('stopwords')
english_sw = stopwords.words('english')



class PostList(list):

    def add_new_arrival(self, sublist):
        self.append(sublist)    #append tuple to self

    def message_count(self):
        return len(self)
    
#global postinglist
#postlinglist=PostList()

#postinglist = [[] for i in range(len(tokenwords))] #initialising an empty list of list which will be our posting list



In [82]:
#Dataset
df3 = pd.read_csv('Quran copy.csv')
df3

,Chapter,Verse,Translation,Tafsir,Combined,Adjusted,Text2,Text
0,1,1,"In the name of Allāh, the Entirely Merciful, ...","1. In the Name of Allah, the Most Gracious, th...",NaN,0.0,In the Name of Allah the Most Gracious the Mo...,In the Name of Allah the Most Gracious the Mo...
1,1,2,"[All] praise is [due] to Allāh, Lord of the w...","2. Al-Hamd be to Allah, the Lord of all that e...",NaN,0.0,Al Hamd be to Allah the Lord of all that exis...,Al Hamd be to Allah the Lord of all that exis...
2,1,3,"The Entirely Merciful, the Especially Merciful,","3. Ar-Rahman (the Most Gracious), Ar-Rahim (th...",NaN,0.0,Ar Rahman the Most Gracious Ar Rahim the Most...,Ar Rahman the Most Gracious Ar Rahim the Most...
3,1,4,Sovereign of the Day of Recompense.,4. The Owner of the Day of Recompense.\n\n \n\...,NaN,0.0,The Owner of the Day of Recompense Indicating...,The Owner of the Day of Recompense Indicating...
4,1,5,It is You we worship and You we ask for help.,"5. You we worship, and You we ask for help.\n\...",NaN,0.0,You we worship and You we ask for help The Li...,You we worship and You we ask for help The Li...
...,...,...,...,...,...,...,...,...
6231,114,2,"The Sovereign of mankind,",NaN,NaN,NaN,NaN,NaN
6232,114,3,"The God of mankind,",NaN,NaN,NaN,NaN,NaN
6233,114,4,From the evil of the retreating whisperer -,NaN,NaN,NaN,NaN,NaN
6234,114,5,Who whispers [evil] into the breasts of manki...,NaN,NaN,NaN,NaN,NaN


# Construction of Posting List and Inverted Index

In [104]:
#INDEX FUNCTION
def index(filename):
    
    print("Reading file...")
    
    global df
    global new_df
    
    df = pd.read_csv(filename)
    
    #print(df)
    
    if (subset_size == 0):
        new_df = df
    else:
        new_df = df[0:subset_size]
    print(new_df)
    
    #Preprocessing and adding new column "Clean Tweets" which will form our searchable tokens and "Original Text", which will preserve our text in string form.
    print("Removing NEWLINE and TAB")
    new_df['Text'] = new_df['Tafsir'].str.replace(r'\d+','')
    new_df["Text"] = new_df['Tafsir'].str.replace('NEWLINE','')
    new_df["Text"] = new_df['Tafsir'].str.replace('TAB','')
    new_df["Text"] = new_df['Tafsir'].str.replace('_','')
    
    
    print("Removing Punctuations")
    new_df["Text"] = new_df['Text'].str.replace('[^\w\s]','')
    new_df["Text"] = new_df['Text'].str.replace('\n','')
    new_df['Text']= new_df['Text'].str.lower()
    
    print("Removing stopwords")
    new_df['Clean_Text'] = new_df['Text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (english_sw)]))
    
    print("Tokenizing and creating new columns...")
    new_df['Original_Text']=new_df['Text']
    new_df['Text']=new_df['Text'].apply(word_tokenize)
    new_df['Clean_Text']=new_df['Clean_Text'].apply(word_tokenize)
    
    
    #Initialising global precedence to some important variables
    global counts
    global tokendict
    global tokendict1
    global tokenwords
    global postsizes
    
    tokenslist = new_df['Clean_Text'].to_list() #creating a list of list out of tweet tokens
    flatlist = list(np.concatenate(tokenslist)) #creating a 1D list to count all unique tokens
    counts = Counter(flatlist) #counting tokens
    
    tokendict = dict(counts) #creating a dictionary of key=token and value=occurences
    
    tokenwords = sorted(list(tokendict.keys())) #extracting only tokens from the dict and making a list of all token words in alphabetical order. This will also act as a hashmap to arrange a token to a specific id (index in this case).
    
    inv_dict={} #our soon to be inverted index

    postinglist=[[] for i in range(len(tokenwords))] #initialising an empty list of list which will be our posting list

    
    
    #Creating Posting List :
    '''
    The idea is to iterate over each row in the dataframe. We pick the tokens from the Clean_Text column
    and check the tokens of the original tweet to see if a token does exist.
    If it does exist, we make note of the index of the dataframe (document number) and see if the posting list already has a value
    for this document number at the position determined by tokenwords. The position determined by tokenwords will determine which of the lists
    of list corresponds to which token and will fill exactly that list whenever a token appears.
    '''
    
    for index, rows in new_df.iterrows():
    
        print('Creating Posting List : %d/100' %(round(index/1000)), end='\r')
    
        for item in new_df['Clean_Text'][index]:
        
            if item in new_df['Text'][index]:
                
                if index not in postinglist[tokenwords.index(item)]:
                    postinglist[tokenwords.index(item)].append(index)
                    
    #Posting List Created!
                    
    postsizes=[len(x) for x in postinglist] #Retrieves size of each of the lists for each token is posting list
    
    zip_iterator = zip(tokenwords, postsizes)


    tokendict1 = dict(zip_iterator) #Creates a dictionary with key = 'token' and value = 'size of posting list for that token'
        
    #NEW LINES
    postlist1 = PostList()
    postlist1.add_new_arrival(postinglist)
    
    #print(postlist1[0][2])
    print("Successfully created posting list!")
    
    
    #Creating Inverted Index
    '''
    The inverted index is now simply created as a dictionary with the following attributes:
    key = 'token' extracted from tokendict1(our zipped dict)
    value = a list of lists with the firstlist having one element equal to the 'size' of the posting list taken from postsizes and
            the second list having all the postings taken from postinglist.
    '''
    
    i=0
    for word in tokenwords:
        if word in tokendict:
            print('Building Inverted Index : %d/100' %(round((i/220540)*100)), end='\r')
            inv_dict[word]=[]
            # append the new number to the existing array at this slot
            inv_dict[word].append(tokendict1[word])
            #NEW LINE
            inv_dict[word].append(postlist1[0][tokenwords.index(word)])
        
            i=i+1
    
    print("Successfully Built Inverted Index")
                  
    return (inv_dict, postinglist)





# Our Query Function to Return Results From Inverted Index

In [127]:
#QUERY FUNCTION which takes two terms, by default term2=None in case only one argument is passed
def newest_query(termlist, explanation):
    
    
    #If only one argument is passed.
    if len(termlist)==1:
        chapverlist = []
        term1 = termlist[0]
        #print("\nPostings List for '" + term1 + "' is as follows :")
        #print(postlist[tokenwords.index(term1)])
        if term1 in tokenwords:
            for i in postlist[tokenwords.index(term1)]:
                if explanation==1:
                    print("\nChapter : " + str(new_df['Chapter'].iloc[i]))
                    print("VerseNo. : " + str(new_df['Verse'].iloc[i]))
                    print("\nVerse : " + str(new_df['Translation'].iloc[i]))
                    print("\nTafsir :" + str(new_df['Tafsir'].iloc[i]))
                    print("--------------------------------------------------")
                    
                chapver = '0'+ str(new_df['Chapter'].iloc[i]) + str(new_df['Verse'].iloc[i])
                chapverlist.append(chapver)
                
            
            return chapverlist #returns the postlist result for the specific token as determined by tokenwords (our basic hashmap) index for that word.
        else:
            return chapverlist
    
    
        
    else: 
        chapverlist=[]
        
        collectedpostlists = []
        
        for i in range (0, len(termlist)):
            #if postlist[tokenwords.index(termlist[i])]:
            if termlist[i] in tokenwords:
                collectedpostlists.append(postlist[tokenwords.index(termlist[i])])
            else:
                continue
            
                        
                        
        
        #collectedpostlists.append(postlist[tokenwords.index(termlist[i])])
            
            
        #postlist_t1 = postlist[tokenwords.index(term1)] #Retreive postlist of term1
        #postlist_t2 = postlist[tokenwords.index(term2)] #Retrieve postlist of term2
    
        #Get lengths of postlists
        #len_t1 = len(postlist_t1) 
        #len_t2 = len(postlist_t2)
    
        #Create an iterator for postlists
        #iter_t1 = iter(postlist_t1)
        #iter_t2 = iter(postlist_t2)
        
        if collectedpostlists:
            intersectlist=list(set.intersection(*[set(x) for x in collectedpostlists]))  #Our soon to be intersection of both postlists.
        else:
            return chapverlist
        
        #print(intersectlist)
        
        #print(intersectlist)
        #Initialising iterators with first value of both postlists
        #i1 = next(iter_t1) 
        #i2 = next(iter_t2)
        
        
        #PostList Intersection Logic
        '''
        We first check if both the iterators represent the same term. If they do, they get added to the intersection list.
        If not, the iterator with the smaller value goes to its next posting. This continues till both lists are exhausted.
        '''
        '''
        for i in range(0, len_t1+len_t2):
            if (i1==i2):
                intersectlist.append(i1)
                i1=next(iter_t1,0)
                i2=next(iter_t2,0)
            elif (i1>i2):
                i2=next(iter_t2,0)
            elif(i2>i1):
                i1=next(iter_t1,0)
        '''
        
        #PostList Intersection Completed
                    
        #print("\nIntersection List for '" + term1 +"' and '" + term2 +"' is as follows :")
        #print(intersectlist)
        
        #print("\nExamples : ")
        
        samples = new_df[['Chapter', 'Verse', 'Translation','Tafsir']].iloc[intersectlist]
        
            
        if (len(intersectlist)!=0):
            
            for i in range(0,len(intersectlist)):
                if explanation==1:
                    print("\nChapter : " + str(samples['Chapter'].iloc[i]))
                    print("VerseNo. : " + str(samples['Verse'].iloc[i]))
                    print("\nVerse :\n " + str(samples['Translation'].iloc[i]))
                    print("\nTafsir :\n" + str(samples['Tafsir'].iloc[i]))
                    print("--------------------------------------------------")
                
                chapver = '0'+ str(samples['Chapter'].iloc[i]) + str(samples['Verse'].iloc[i])
                chapverlist.append(chapver)
            
                #if(i==samples_intersect):
                #    break
        
        return(chapverlist)





# MAIN

In [106]:
global postlist 

result=[]

tic = time.perf_counter()
invertedindex, postlist = index('Quran.csv') #postlist stores the posting list, invertedindex stores the complete inverted index.


#Printing first n elements of each (samples_invindex variable under IMPORTANT VARIABLES) and showing how index matching is synchronous (hashmap between tokenwords and posting list is a success).
if (samples_invindex==0):
    print("Full inverted index:")
    print(inverted_index)
else:
    print("\n First %d elements of inverted index" %samples_invindex)
    n_items = more_itertools.take(samples_invindex, invertedindex.items())
    print(n_items)
        
if (samples_postlist == 0):
    print("Full postings list:")
    print(postlist)
else:
    print("\nFirst %d elements of posting list" %samples_postlist)
    print(postlist[:samples_postlist])
    
    
    
#result = query( "nation") #Returns complete postlist for a single term
#print(result)

#result = query( "sin", "forgive") #Returns intersection of postlist of both terms.

toc = time.perf_counter()
print(f"Completed in {toc - tic:0.4f} seconds")

    




Reading file...
      Chapter  Verse                                        Translation  \
0           1      1   In the name of Allāh, the Entirely Merciful, ...   
1           1      2   [All] praise is [due] to Allāh, Lord of the w...   
2           1      3    The Entirely Merciful, the Especially Merciful,   
3           1      4                Sovereign of the Day of Recompense.   
4           1      5      It is You we worship and You we ask for help.   
...       ...    ...                                                ...   
1132        7    179   And We have certainly created for Hell many o...   
1133        7    180   And to Allāh belong the best names, so invoke...   
1134        7    181  And among those We created is a community whic...   
1135        7    182  But those who deny Our signs - We will progres...   
1136        7    183   And I will give them time. Indeed, My plan is...   

                                                 Tafsir  
0     1. In the Name of A

# Example Query

In [130]:

explanation=0  #change this to 1, to get verse explanations as well.
query = 'Day of Uhud'  #Enter your query here

def boolean_retrieval(query):
    print("Query :", query)
    query=query.lower()
    tokens = re.findall(r'([A-Za-z]+)',query)
    #print(tokens)

    accepted_pos = ['VBN','VBG','VB','RP','RBS','RBR','RB','NNS','NNP','NN','JJS','JJR','JJ']

    query_words = nltk.pos_tag(tokens)


    query_list =[]

    for i in range(0,len(query_words)):
        if (query_words[i][1] in accepted_pos) and (query_words[i][0]!='story') and (query_words[i][0]!='be') and (query_words[i][0]!='i') :
            query_list.append(query_words[i][0])
    
    #print(query_list)

    chapvers = newest_query(query_list, explanation)
    
    return chapvers



chapverses = boolean_retrieval(query)

for i in range(0,len(chapverses)):
    print("Quran " + "(" + chapverses[i][:2] + ":" + chapverses[i][2:] + ")")
    

    

Query : Day of Uhud
Quran (03:121)
Quran (03:122)
Quran (03:123)
Quran (03:124)
Quran (03:125)
Quran (03:126)
Quran (03:127)
Quran (03:128)
Quran (03:149)
Quran (03:150)
Quran (03:151)
Quran (03:152)
Quran (03:153)
Quran (03:154)
Quran (03:155)
Quran (07:8)
Quran (07:9)
Quran (03:159)
Quran (03:160)
Quran (03:161)
Quran (03:162)
Quran (03:163)
Quran (03:164)
Quran (03:165)
Quran (03:166)
Quran (06:55)
Quran (06:56)
Quran (06:57)
Quran (06:58)
Quran (06:59)
Quran (03:167)
Quran (03:168)
Quran (03:169)
Quran (03:170)
Quran (03:171)
Quran (02:74)
Quran (03:172)
Quran (03:173)
Quran (03:174)
Quran (03:175)
Quran (03:176)
Quran (03:177)
Quran (03:178)
Quran (03:179)
Quran (03:180)
Quran (04:5)
Quran (04:6)
Quran (04:141)


# Evaluation

In [131]:
df = pd.read_csv('AnnotationQueries.csv')

In [132]:
df['Mak'] = df['Mak'].str.strip('[]').str.split(',')
df['Ufkun'] = df['Ufkun'].str.strip('[]').str.split(',')
df['System'] = df['System'].str.strip('[]').str.split(',')

In [133]:
df

,Unnamed: 0,Query,Ufkun,Mak,System
0,0,What are the rules of fasting?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,1,Was the qiblah changed?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0..."
2,2,Story of Saul,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0..."
3,3,What are the characteristics of the hypocrites?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
4,4,Why was Adam created?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
5,5,How to get the support of Allah?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
6,6,Story of Moses and Fira'un,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0..."
7,7,Story of the Children of Israel and the Cow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
8,8,What are the Proofs of Prophet Muhammad's Prop...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
9,9,What are the manners of speech?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


# Cohen's Kappa for Individual Queries

In [136]:

def cohen_kappa(ann1, ann2):

    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  

    uniq = set(ann1 + ann2)
    E = 0  
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

In [137]:
#Code for Cohen's Kappa
from statistics import mean,median

agree_c = []

df_kappa = pd.DataFrame(columns = ['Query', 'Kappa'], index=range(57)) 
df_kappa = df_kappa.fillna(0)

for index,rows in df.iterrows():
    
    ann1 = df['Mak'].iloc[index]  #read the list of Mak's annotation for this index
    ann2 = df['Ufkun'].iloc[index] #read the list of Ufkun's annotation for this index
    ann3 = df['System'].iloc[index] #read the list of System's annotation for this index
    
    
    
    ann1 = [int(x) for x in ann1] #making all those list values to int
    ann2 = [int(x) for x in ann2]
    ann3 = [int(x) for x in ann3]
    
    if '1' not in ann1 :  #if list is empty, it has all zeroes, K-alpha needs atleast two symbols. So I put 1 in the last element just for this.
        ann1[-1] = 1
    if '1' not in ann2 :
        ann2[-1] = 1
    if '1' not in ann3 :
        ann3[-1] = 1
    
    
    kappa = cohen_kappa(ann2, ann1)
    agree_c.append(kappa)
    
    df_kappa['Query'].loc[index] = df['Query'][index]
    df_kappa['Kappa'].loc[index] = kappa
    
    #print(cohen_kappa(ann1,ann2))
    
df_kappa

,Query,Kappa
0,What are the rules of fasting?,0.5982
1,Was the qiblah changed?,1.0000
2,Story of Saul,0.9087
3,What are the characteristics of the hypocrites?,0.5965
4,Why was Adam created?,0.8316
5,How to get the support of Allah?,0.8563
6,Story of Moses and Fira'un,0.6353
7,Story of the Children of Israel and the Cow,1.0000
8,What are the Proofs of Prophet Muhammad's Prop...,0.8563
9,What are the manners of speech?,1.0000


In [140]:
round(mean(agree_c),2)

0.81

# Evaluating Against System Results

In [141]:
#Individual query prec,recall,f1
from statistics import mean,median, mode

df_eval = pd.DataFrame(columns = ['Query', 'Prec', 'Recall', 'F1'], index=range(57)) 
df_eval = df_eval.fillna(0)
#print(df_eval)

agree_c = []
preclist=[]
reclist=[]
f1_list=[]

for index,rows in df.iterrows():
    
    ann1 = df['Mak'].iloc[index]  #read the list of Mak's annotation for this index
    ann2 = df['Ufkun'].iloc[index] #read the list of Ufkun's annotation for this index
    ann3 = df['System'].iloc[index] #read the list of System's annotation for this index
    
    
    
    ann1 = [int(x) for x in ann1] #making all those list values to int
    ann2 = [int(x) for x in ann2]
    ann3 = [int(x) for x in ann3]
    
    if '1' not in ann1 :  #if list is empty, it has all zeroes, K-alpha needs atleast two symbols. So I put 1 in the last element just for this.
        ann1[-1] = 1
    if '1' not in ann2 :
        ann2[-1] = 1
    if '1' not in ann3 :
        ann3[-1] = 1
        
    ann12 = [0]*len(ann1)
    for i in range(len(ann2)):

        if ann1[i] != ann2[i]:
            ann12[i] = 1
        else:
            ann12[i] = ann2[i]
        

    correct = ann12
    predicted = ann3
    
    tp=0
    tn=0
    fp=0
    fn=0
    
    for i in range(len(ann12)):
        if correct[i] == 1 and predicted[i]==0:
            fn+=1
        elif correct[i] == 0 and predicted[i]==1:
            fp+=1
        elif correct[i] == 0 and predicted[i]==0:
            tn+=1
        elif correct[i] == 1 and predicted[i]==1:
            tp+=1
    
    prec=0
    rec=0
    f1=0
    
    prec = round((tp)/(tp+fp),2)
    preclist.append(prec)
    #print(prec)

    rec = round((tp)/(tp+fn),2)
    reclist.append(rec)
    #print(rec)

    f1 = round((2*prec*rec)/(prec+rec),2)
    f1_list.append(f1)
    #print(f1)
    
    #print('Prec: ', prec, 'Recall : ', rec, 'F1: ', f1)
    
    df_eval['Query'].loc[index] = df['Query'][index]
    df_eval['Prec'].loc[index] = prec
    df_eval['Recall'].loc[index] = rec
    df_eval['F1'].loc[index] = f1
    
    
    
    
    #agree_c.append(cohen_kappa(ann1, ann2))
    #print(cohen_kappa(ann1,ann2))

df_eval

,Query,Prec,Recall,F1
0,What are the rules of fasting?,1.00,0.43,0.60
1,Was the qiblah changed?,0.21,1.00,0.35
2,Story of Saul,0.33,0.50,0.40
3,What are the characteristics of the hypocrites?,0.11,0.07,0.09
4,Why was Adam created?,0.33,0.21,0.26
5,How to get the support of Allah?,0.11,0.12,0.11
6,Story of Moses and Fira'un,0.11,0.03,0.05
7,Story of the Children of Israel and the Cow,0.11,0.11,0.11
8,What are the Proofs of Prophet Muhammad's Prop...,0.22,0.25,0.23
9,What are the manners of speech?,0.11,1.00,0.20


In [142]:
print("Mean Precision :",mean(preclist))
print("Mean Recall : ", mean(reclist))
print("Mean F1:", mean(f1_list))

print("\n")
print("Median Precision :", median(preclist))
print("Median Recall :", median(reclist))
print("Median F1 :", median(f1_list))

Mean Precision : 0.44333333333333336
Mean Recall :  0.42964912280701756
Mean F1: 0.3392982456140351


Median Precision : 0.22
Median Recall : 0.33
Median F1 : 0.28
